In [1]:
import itemQuery as iq
import articleParse as ap
from string import ascii_letters, digits

import psycopg2
import config as config

newsFeedTypes = ['News', 'All News', 'news', 'TeamNews', 'Latest Content', 'News Stories', 'Team Stories', 'Latest Headlines', 'Articles']

In [2]:
def ExtractAlphanumeric(InputString):
    return "".join([ch for ch in InputString if ch in (ascii_letters + digits + ' ' )])

def parseForCorpus(link):
    article = iq.getArticle(link)
    
    articleParser = ap.articleContentParser()
    articleParser.feed(article)
    article1 = articleParser.getContent()
    
    if not article1:
        # switch to pane parseing
        paneParser = ap.paneContentParser()
        paneParser.feed(article)
        article1 = paneParser.getContent()
    
    # Alpha Numeric Cleaning
    clean1 = ExtractAlphanumeric(article1)
    
    sp1 = clean1.split(' ')
    
    # Space and Link Cleaning
    sp2 = [elm for elm in sp1 if elm] # Spaces Longer than one character
    sp3 = [elm for elm in sp2 if 'http' not in elm] # No links
    noWords = len(sp3)
    
    clean2 = ' '.join(sp3)
    
    return clean2, noWords

In [15]:
newsTypeConditions = 'type = ' + " OR type = ".join(["\'" + t + "\'" for t in newsFeedTypes])

In [26]:
# initial query
conn = config.connect()
cursor = conn.cursor()
cursor.execute("SELECT id, link, date FROM nfl_team_articles WHERE %s" % newsTypeConditions)
newsLinks = (cursor.fetchall())

In [32]:
# automated query

lastId = 450

conn = config.connect()
cursor = conn.cursor()
cursor.execute("SELECT id, link, date FROM nfl_team_articles WHERE (id > %s) AND (%s)" % (lastId,newsTypeConditions))
newsLinks = (cursor.fetchall())

In [33]:
newsLinks

[(451,
  'http://www.buccaneers.com/news/article-smith/Enhanced-Raymond-James-Stadium-Ready-to-Shine/e95d5e38-b214-4128-aee4-4d1f7197266f',
  'Thu, 24 Aug 2017 21:05:08 GMT'),
 (453,
  'http://www.packers.com/news-and-events/article-special-feature/article-1/Ha-Ha-Clinton-Dix-on-a-mission-to-make-a-difference/62e6b7a9-dac4-4042-9901-947511a7cb85',
  'Thu, 24 Aug 2017 21:00:08 GMT'),
 (456,
  'http://www.houstontexans.com/news/article-2/24-observations-Texans-Saints-joint-practice/a526ae6f-1cac-481a-8e17-76c18ad7f4b2',
  'Thu, 24 Aug 2017 20:59:34 GMT'),
 (458,
  'http://www.vikings.com/news/article-1/3-Observations-Vikings-Prepare-for-Preseason-Debut-at-US-Bank-Stadium/0eb0adf9-cc0c-48c0-b5f3-802b5aaffa8a',
  'Thu, 24 Aug 2017 20:57:00 GMT'),
 (459,
  'http://www.therams.com/news-and-events/article-1/TRANSCRIPT-McVay-Gurley-8-24-Press-Conference/2e7d8d0f-a853-4201-ac20-60f2084641af',
  'Thu, 24 Aug 2017 20:53:23 GMT'),
 (460,
  'http://www.steelers.com/news/article-1/Just-enough-time-f